In [23]:
import pandas as pd
# Imports CSV
df = pd.read_csv('../raw_data/constituency_raw.csv')
# Reduces CSV to relevant election years
filtered_df = df[df['Election Year'].isin([2005, 2010, 2015, 2017, 2019])]
#Creates new index
filtered_df['new_index'] = filtered_df['Election Year'].astype(str) + '_' + filtered_df['Constituency'] + '_' + filtered_df['Boundary Year'].astype(str)
# Pivots table for the first time
pivot_votes_df = filtered_df.pivot_table(
    index=['Election Year', 'Boundary Year', 'Constituency', 'Candidate','Party', 'new_index'],
    values='Votes',
    fill_value=0
).reset_index()

/tmp/ipykernel_3182/2043912932.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df['new_index'] = filtered_df['Election Year'].astype(str) + '_' + filtered_df['Constituency'] + '_' + filtered_df['Boundary Year'].astype(str)


In [25]:
# Reduces CSV to relevant parties
parties_cleaned = pivot_votes_df.reset_index(drop=False)['Party'].apply(lambda x: x if x in ['British National Party', 'The Brexit Party', 'Conservative', 'Green Party', 'Liberal Democrats', 'Labour', 'Plaid Cymru', 'SNP', 'UK Independence Party (UKIP)'] else 'Other')
pivot_votes_df['Party_cleaned'] = parties_cleaned


In [26]:
#Drops irrelevant columns
pivot_votes_df.drop(columns='Party', inplace=True)
pivot_votes_df.set_index('new_index', inplace=True)


In [27]:
# Makes second pivot
pivot_table = pd.pivot_table(pivot_votes_df,
                             index=['new_index'],
                             columns=['Party_cleaned'],
                             values='Votes',
                             aggfunc='sum',
                             fill_value=0)

In [32]:
# Renames columns to follow our style
rename_dict = {
    'British National Party': 'NAT_ACTUAL_CONS',
    'The Brexit Party': 'BRX_ACTUAL_CONS',
    'Conservative': 'CON_ACTUAL_CONS',
    'Green Party': 'GRE_ACTUAL_CONS',
    'Liberal Democrats': 'LIB_ACTUAL_CONS',
    'Labour': 'LAB_ACTUAL_CONS',
    'Plaid Cymru': 'PLC_ACTUAL_CONS',
    'SNP': 'SNP_ACTUAL_CONS',
    'UK Independence Party (UKIP)': 'UKI_ACTUAL_CONS',
    'Other': 'OTH_ACTUAL_CONS'
}
pivot_table.rename(columns=rename_dict, inplace=True)

In [33]:
#Readds relevant columns
pivot_table['Election Year'] = pivot_table.index.str.split('_').str[0].astype(int)
pivot_table['Constituency'] = pivot_table.index.str.split('_').str[1]
pivot_table['Boundary Year'] = pivot_table.index.str.split('_').str[2]

In [34]:
# Reorders columns
columns_reordered = ['Constituency', 'Election Year', 'Boundary Year',
                     'CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS',
                     'LIB_ACTUAL_CONS', 'NAT_ACTUAL_CONS', 'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS',
                     'SNP_ACTUAL_CONS', 'BRX_ACTUAL_CONS', 'UKI_ACTUAL_CONS']
df_final = pivot_table[columns_reordered]

In [35]:
# Calculates total votes
df_final['total_votes'] = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS',
                  'NAT_ACTUAL_CONS', 'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS', 'SNP_ACTUAL_CONS', 'BRX_ACTUAL_CONS',
                  'UKI_ACTUAL_CONS']].sum(axis=1)

/tmp/ipykernel_3182/4202756632.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['total_votes'] = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS',


In [36]:
# Switches to percentage
df_percentage = df_final[['CON_ACTUAL_CONS', 'GRE_ACTUAL_CONS', 'LAB_ACTUAL_CONS', 'LIB_ACTUAL_CONS', 'NAT_ACTUAL_CONS',
                    'OTH_ACTUAL_CONS', 'PLC_ACTUAL_CONS', 'SNP_ACTUAL_CONS', 'BRX_ACTUAL_CONS',
                    'UKI_ACTUAL_CONS']].div(df_final['total_votes'], axis=0) * 100

In [37]:
df_result = pd.concat([df_final[['Election Year', 'Constituency']], df_percentage], axis=1)


In [38]:
df_result.rename(columns={'Election Year': 'election_year'}, inplace=True)

In [39]:
df_result

Party_cleaned,election_year,Constituency,CON_ACTUAL_CONS,GRE_ACTUAL_CONS,LAB_ACTUAL_CONS,LIB_ACTUAL_CONS,NAT_ACTUAL_CONS,OTH_ACTUAL_CONS,PLC_ACTUAL_CONS,SNP_ACTUAL_CONS,BRX_ACTUAL_CONS,UKI_ACTUAL_CONS
new_index,,,,,,,,,,,,
2005_Aberavon_2005,2005,Aberavon,10.178049,1.694127,60.048499,13.752325,0.0,2.551156,11.775844,0.000000,0.000000,0.0
2005_Aberdeen North_2005,2005,Aberdeen North,9.433859,0.000000,42.466015,23.917672,0.0,1.886226,0.000000,22.296228,0.000000,0.0
2005_Aberdeen South_2005,2005,Aberdeen South,17.140386,0.000000,36.693016,33.454266,0.0,2.813484,0.000000,9.898849,0.000000,0.0
2005_Aberdeenshire West and Kincardine_2005,2005,Aberdeenshire West and Kincardine,28.366308,0.000000,13.133884,46.304745,0.0,0.910008,0.000000,11.285056,0.000000,0.0
2005_Airdrie and Shotts_2005,2005,Airdrie and Shotts,9.864889,0.000000,59.014416,11.436154,0.0,3.145546,0.000000,16.538995,0.000000,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...
2019_Yeovil_2010,2019,Yeovil,58.366520,2.748903,6.346608,31.061424,0.0,1.476544,0.000000,0.000000,0.000000,0.0
2019_Ynys Mon_2010,2019,Ynys Mon,35.453600,0.000000,30.069490,0.000000,0.0,0.000000,28.501860,0.000000,5.975049,0.0
2019_York Central_2010,2019,York Central,27.809312,4.256136,55.170185,8.380972,0.0,1.395819,0.000000,0.000000,2.987577,0.0


In [40]:
# Imports national election results to concatenante
election_results_df = pd.read_csv('../processed_data/election_results_dataframe.csv')
election_results_df.rename(columns={'Year': 'election_year'}, inplace=True)

In [41]:
election_results_df.rename(columns={'Year': 'election_year'}, inplace=True)

In [42]:
# Merges national onto constituency election results
cons_result_comp = df_result.merge(election_results_df, on='election_year', how='left')

In [43]:
# Renames national result columns
rename_dict2 = {
    'BRX_ACTUAL_PERCENTAGE': 'BRX_NAT_RESULT',
    'CON_ACTUAL_PERCENTAGE': 'CON_NAT_RESULT',
    'GRE_ACTUAL_PERCENTAGE': 'GRE_NAT_RESULT',
    'LIB_ACTUAL_PERCENTAGE': 'LIB_NAT_RESULT',
    'LABOUR_ACTUAL_PERCENTAGE': 'LAB_NAT_RESULT',
    'NAT_ACTUAL_PERCENTAGE': 'NAT_NAT_RESULT',
    'PLC_ACTUAL_PERCENTAGE': 'PLC_NAT_RESULT',
    'SNP_ACTUAL_PERCENTAGE': 'SNP_NAT_RESULT',
    'UKI_ACTUAL_PERCENTAGE': 'UKI_NAT_RESULT',
    'OTH_PERCENTAGE': 'OTH_NAT_RESULT'
}
cons_result_comp.rename(columns=rename_dict2, inplace=True)


In [44]:
# Drops and renames columns
cons_result_comp.drop(columns=['Geography', 'Country'], inplace=True)
cons_result_comp.rename(columns={'CON_ACT_CONS': 'CON_ACTUAL_CONS'}, inplace=True)
cons_result_comp.rename(columns={'BRX_ACT_CONS': 'BRX_ACTUAL_CONS'}, inplace=True)

In [49]:
# Calculates bias score
for index, row in cons_result_comp.iterrows():
    bias_scores = {}
    # Loop through each party
    for party in ['CON', 'GRE', 'LAB', 'LIB', 'NAT', 'OTH', 'PLC', 'SNP', 'BRX', 'UKI']:
        # Calculate the absolute difference between the constituency result and national result for each party
        bias_scores[party] = (row[f'{party}_ACTUAL_CONS'] - row[f'{party}_NAT_RESULT'])
    # Assign the bias scores to the respective columns
    for party, score in bias_scores.items():
        cons_result_comp.loc[index, f'{party}_Bias_Score'] = score

In [51]:
# Filters datafra,e
biases_df = cons_result_comp.filter(like='Bias')

In [52]:
# Calculates bias per constituency
average_biases_per_constituency = biases_df.groupby(cons_result_comp['Constituency']).mean()

In [53]:
average_biases_per_constituency

,CON_Bias_Score,GRE_Bias_Score,LAB_Bias_Score,LIB_Bias_Score,OTH_Bias_Score,PLC_Bias_Score,SNP_Bias_Score,BRX_Bias_Score,UKI_Bias_Score,NAT_Bias_Score
Constituency,,,,,,,,,,
Aberavon,-23.307236,-0.858548,23.223869,-6.440048,-0.733584,8.918441,-2.965755,1.564677,0.300333,0.297851
Aberconwy,2.292982,-1.576494,0.884625,-4.312122,-3.405292,11.439902,-3.327539,-0.503170,-1.012279,-0.480612
Aberdeen North,-22.896858,-1.933300,-2.132344,-2.494376,-2.238113,-0.556518,36.282878,0.136314,-3.978052,-0.189630
Aberdeen South,-10.489169,-1.933300,-7.552604,2.300544,-2.417282,-0.556518,24.940048,-0.402536,-3.608543,-0.280639
Aberdeenshire West and Kincardine,-2.645865,-1.933300,-23.964078,10.929087,-3.136656,-0.556518,25.447191,-0.402536,-3.437849,-0.299475
...,...,...,...,...,...,...,...,...,...,...
Ynys Mon,-14.653125,-1.933300,0.868470,-10.812434,0.853511,28.183887,-2.965755,0.792474,0.192765,-0.526491
York Central,-11.617521,2.294539,17.793366,-0.908316,-2.784202,-0.534640,-3.327539,0.243724,-1.308598,0.149188
York Outer,8.438650,-0.516706,-5.411639,6.557452,-3.207015,-0.534640,-3.327539,-0.503170,-1.463186,-0.032207


In [54]:
 average_biases_per_constituency.to_csv('../raw_data/constituency_bias_dataframe.csv', index=True)